In [228]:
using JuMP, Ipopt, Gurobi
using DataFrames

In [268]:
scenario_no = 4;

In [269]:
T = 3;
S = [50,50,50];
α_s = [5,20,10];
D = [25,100,25];
α_d = [30,60,40];
α_sc = [0.1 for i in 1:T]
α_sd = [0.1 for i in 1:T]
SOCmin = 0;
SOCmax = 100;
Pmin = 0;
Pmax = 10;

if scenario_no == 1
    R = 25; SOC0 = 50;
elseif scenario_no == 2
    R = 15; SOC0 = 50;
elseif scenario_no == 3
    R = 15; SOC0 = 95;
elseif scenario_no == 4
    R = 5; SOC0 = 50;
end

η_s = 1.0;
η_c = 0.9;
η_d = 0.8;

In [270]:
s_lb = [SOCmin-SOC0 for _ in 1:T]
s_lb[T] = 0
s_ub = SOCmax-SOC0
optimizer = optimizer_with_attributes(Gurobi.Optimizer, "OutputFlag" => 0)

m = Model(optimizer)

@variable(m, p[1:T] >= 0)
@variable(m, d[1:T] >= 0)
@variable(m, pc[1:T] >= 0)
@variable(m, pd[1:T] >= 0)

# balance constraint
bals = Dict()
for t in 1:T
    bals[t] = @constraint(m, p[t] + pd[t] - pc[t] - d[t] == 0)
end

# SOC update
soc_lb = Dict()
soc_ub = Dict()
for t in 1:T
    soc_lb[t] = @constraint(m, η_c * sum(pc[1:t]) - 1/η_d * sum(pd[1:t]) >= s_lb[t])
    soc_ub[t] = @constraint(m, -η_c * sum(pc[1:t]) + 1/η_d * sum(pd[1:t]) >= -s_ub)
end

# Charging/discharging bound
charge_bound = Dict()
for t in 1:T
    charge_bound[t] = @constraint(m, -pc[t] - pd[t] >= -Pmax)
end

# Generator ramping
for t in 1:T-1
    @constraint(m, p[t+1] - p[t] <= R)
    @constraint(m, p[t+1] - p[t] >= -R)
end

# variable bounds
@constraint(m, d .<= D)
@constraint(m, p .<= S)

# objective: social welfare
@objective(m, Min, sum(α_s .* p) - sum(α_d .* d) + sum(α_sd .* pd) + sum(α_sc .* pc));

Set parameter Username
Academic license - for non-commercial use only - expires 2022-02-11


In [271]:
optimize!(m)
println("Objective value: ", objective_value(m))

df = DataFrame()
df.prices = [dual(bals[t]) for t in 1:T]
df.pc = value.(pc)
df.pd = value.(pd)
df.d = value.(d)
df.p = value.(p)
df.soc = [SOC0 + value(η_c * sum(pc[1:t]) - 1/η_d * sum(pd[1:t])) for t in 1:T]
display(df)

,prices,pc,pd,d,p,soc
,Float64,Float64,Float64,Float64,Float64,Float64
1,-24.9,10.0,0.0,25.0,35.0,59.0
2,60.0,0.0,10.0,50.0,40.0,46.5
3,-0.1,10.0,0.0,25.0,35.0,55.5


Objective value: -3422.0


In [272]:
s_lb = [SOCmin-SOC0 for _ in 1:T]
s_lb[T] = 0
s_ub = SOCmax-SOC0
optimizer = optimizer_with_attributes(Gurobi.Optimizer, "OutputFlag" => 0)

m = Model(optimizer)

@variable(m, p[1:T] >= 0)
@variable(m, d[1:T] >= 0)
@variable(m, pc[1:T] >= 0)
@variable(m, pd[1:T] >= 0)

# balance constraint
bals = Dict()
for t in 1:T
    bals[t] = @constraint(m, p[t] + pd[t] - pc[t] - d[t] == 0)
end

# SOC update
soc_lb = Dict()
soc_ub = Dict()
for t in 1:T
    soc_lb[t] = @constraint(m, η_c * sum(pc[1:t]) - 1/η_d * sum(pd[1:t]) >= s_lb[t])
    soc_ub[t] = @constraint(m, η_c/η_d * (sum(pd[1:t])-sum(pc[1:t])) >= -s_ub)
end

# Charging/discharging bound
charge_bound = Dict()
for t in 1:T
    charge_bound[t] = @constraint(m, -pc[t] - pd[t] >= -Pmax)
end

# Generator ramping
for t in 1:T-1
    @constraint(m, p[t+1] - p[t] <= R)
    @constraint(m, p[t+1] - p[t] >= -R)
end

# variable bounds
@constraint(m, d .<= D)
@constraint(m, p .<= S)

# objective: social welfare
@objective(m, Min, sum(α_s .* p) - sum(α_d .* d) + sum(α_sd .* pd) + sum(α_sc .* pc));

Set parameter Username
Academic license - for non-commercial use only - expires 2022-02-11


In [273]:
optimize!(m)

println("Objective value: ", objective_value(m))

df = DataFrame()
df.prices = [dual(bals[t]) for t in 1:T]
df.pc = value.(pc)
df.pd = value.(pd)
df.d = value.(d)
df.p = value.(p)
df.soc = [SOC0 + value(η_c * sum(pc[1:t]) - 1/η_d * sum(pd[1:t])) for t in 1:T]
display(df)

,prices,pc,pd,d,p,soc
,Float64,Float64,Float64,Float64,Float64,Float64
1,-24.9,10.0,0.0,25.0,35.0,59.0
2,60.0,0.0,10.0,50.0,40.0,46.5
3,-0.1,10.0,0.0,25.0,35.0,55.5


Objective value: -3422.0


In [274]:
# Market clearing formulation with virtual links

s_lb = [SOCmin-SOC0 for _ in 1:T]
s_lb[T] = 0
s_ub = SOCmax-SOC0

vls = unique([(i,j) for i in 1:T for j in 1:T if i != j])
η = η_c * η_d
α_δ = Dict((i,j) => α_sc[i] + η * α_sd[j] for (i,j) in vls)
optimizer = optimizer_with_attributes(Gurobi.Optimizer, "OutputFlag" => 0)

m = Model(optimizer)

@variable(m, p[1:T] >= 0)
@variable(m, d[1:T] >= 0)
@variable(m, δ[vls] >= 0)
@variable(m, pnc[1:T] >= 0)
@variable(m, pnd[1:T] >= 0)

# balance constraint
bals = Dict()
for t in 1:T
    bals[t] = @constraint(m, p[t] + pnd[t] + η * sum(δ[(i,j)] for (i,j) in vls if j == t) - sum(δ[(i,j)] for (i,j) in vls if i == t) - d[t] - pnc[t] == 0)
end

# SOC Bounds
for t in 1:T
    @constraint(m, η_c * sum((sum(δ[(i,j)] for (i,j) in vls if i == s) - sum(δ[(i,j)] for (i,j) in vls if j == s)) for s in 1:t) >= s_lb[t] + 1/η_d * sum(pnd[1:t]))
    @constraint(m, η_c/η_d * sum(sum(δ[(i,j)] for (i,j) in vls if i == s) - η * sum(δ[(i,j)] for (i,j) in vls if j == s) for s in 1:t) <= s_ub - η_c/η_d * sum(pnc[1:t]))
end

# Charging/discharging bound
for t in 1:T
    @constraint(m, Pmin <= pnc[t] + pnd[t] + sum(δ[(i,j)] for (i,j) in vls if i == t) + η * sum(δ[(i,j)] for (i,j) in vls if j == t) <= Pmax)
end 

# variable bounds
@constraint(m, d .<= D)
@constraint(m, p .<= S)

# Generator ramping
for t in 1:T-1
    @constraint(m, p[t+1] - p[t] <= R)
    @constraint(m, p[t+1] - p[t] >= -R)
end

# objective: social welfare
@objective(m, Min, sum(α_s .* p) - sum(α_d .* d) + sum(α_δ[v] * δ[v] for v in vls) + sum(α_sd .* pnd) + sum(α_sc .* pnc));

Set parameter Username
Academic license - for non-commercial use only - expires 2022-02-11


In [275]:
optimize!(m)
println("Objective value: ", objective_value(m))

df = DataFrame()
pc_vals = [value(sum(δ[(i,j)] for (i,j) in vls if i == t) + pnc[t]) for t in 1:T]
pd_vals = [η * value(sum(δ[(i,j)] for (i,j) in vls if j == t)) + value(pnd[t]) for t in 1:T]
df.prices = [dual(bals[t]) for t in 1:T]
df.pc = pc_vals
df.pd = pd_vals
df.d = value.(d)
df.p = value.(p)
df.soc = [SOC0 + value(η_c * sum(pc_vals[1:t]) - 1/η_d * sum(pd_vals[1:t])) for t in 1:T]
display(df)

,prices,pc,pd,d,p,soc
,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.1,10.0,0.0,25.0,35.0,59.0
2,60.0,0.0,10.0,50.0,40.0,46.5
3,-24.9,10.0,0.0,25.0,35.0,55.5


Objective value: -3422.0


In [276]:
for t in 1:T
    if value(pnc[t]) > 0
        println("Net-charging occurs at t = $(t) with power $(value(pnc[t]))")
        println("    Payment: ", value(pnc[t]) * df.prices[t])
    end
    if value(pnd[t]) > 0
        println("Net-discharging occurs at t = $(t) with power $(value(pnd[t]))")
        println("    Remuneration: ", value(pnd[t]) * df.prices[t])
    end
end

Net-charging occurs at t = 1 with power 6.111111111111114
    Payment: -0.6111111111111115
